In [1]:
import torch
from efficientnet_pytorch import EfficientNet
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path

In [8]:
# you can find a pretrained model at model/b3.pth
MODEL_F = './model/b0.pth'
# directory with the numpy optical flow images you want to use for inference
OF_NPY_DIR = './opical-flow-estimation-with-RAFT/demo-output'

In [3]:
# check if cuda is available
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [4]:
device

'cpu'

## Load Model

In [4]:
V = 0     # what version of efficientnet did you use
IN_C = 2  # number of input channels
NUM_C = 1 # number of classes to predict

In [5]:
model = EfficientNet.from_pretrained(f'efficientnet-b{V}', in_channels=IN_C, num_classes=NUM_C)
state = torch.load(MODEL_F,map_location=torch.device('cpu'))
model.load_state_dict(state)
model.to(device);

Loaded pretrained weights for efficientnet-b0


In [6]:
def inference(of_f):
    of = np.load(of_f)
    i = torch.from_numpy(of).to(device)
    pred = model(i)
    del i
    torch.cuda.empty_cache()
    return pred

In [9]:
# loop over all files in directory and predict
for f in Path(OF_NPY_DIR).glob('*.npy'): # glob可以查找符合特定规则的文件路径名
    y_hat = inference(f).item()
    print(f'{f.name}: {round(y_hat, 2)}')

0.npy: 5.49
1.npy: 2.02
2.npy: 4.33
3.npy: 3.76
4.npy: 4.09
5.npy: 5.26
6.npy: 4.31
7.npy: 5.11
8.npy: 3.66
